In [1]:
import pandas as pd
import numpy as np
import faiss
import time

In [17]:
#pd.set_option('display.max_rows', None)

In [48]:
#pd.reset_option('display.max_rows')

In [74]:
#d = pd.read_csv('data/tokens.txt', header=None, delimiter=' ', quoting=3)

In [77]:
#d.to_pickle('data/tokens.pkl')

In [2]:
d = pd.read_pickle('data/tokens.pkl')

In [3]:
#used to test after reading tokens.txt to see if I had done it right
#then used to see what the first ten words were
print(d.iloc[:11, :3])

      0         1         2
0   the  0.306717 -0.320530
1     ,  0.022532  0.320788
2     . -0.075626  0.369955
3    of  0.487177 -0.193218
4    to  0.173940 -0.104660
5   and  0.222883 -0.001161
6    in  0.712315  0.451855
7     a  0.048874  0.379618
8     "  0.019833  0.136752
9     - -0.308798 -0.711716
10   's  0.777245 -0.275744


In [4]:
first_ten_words = d.iloc[[0,3,4,5,6,7,11,12,13,14]]

In [ ]:
#cosine similarity = v1 dot v2 / (norm(v1) * norm(v2))
#cosine distance = 1 - cosine similarity

cos_similarities = np.empty((10,10))
for i in range(10):
    for j in range (10):
        cos_similarities[i, j] = np.dot(first_ten_words.iloc[i, 1:], first_ten_words.iloc[j, 1:]) / (np.linalg.norm(first_ten_words.iloc[i, 1:]) * np.linalg.norm(first_ten_words.iloc[j, 1:]))
cos_similarities = np.clip(cos_similarities, -1, 1) #clipped to fix a rounding error
cos_distances = 1 - cos_similarities

print(np.round(cos_distances, decimals=3))

In [ ]:
#angular distance = arccos(v1 dot v2 / (norm(v1) * norm(v2)))

angular_distances = np.arccos(cos_similarities)

print(np.round(angular_distances, decimals=3))

In [ ]:
ang_dis_arr_rounded_no_repeats = np.tril(np.round(angular_distances, decimals=4))
ang_dis_df = pd.DataFrame(columns=['word1', 'word2', 'angular distance'])
for i in range(10):
    for j in range(10):
        if ang_dis_arr_rounded_no_repeats[i,j] != 0:
            ang_dis_df.loc[len(ang_dis_df)] = {'word1': first_ten_words.iloc[i,0], 'word2': first_ten_words.iloc[j,0], 'angular distance': ang_dis_arr_rounded_no_repeats[i,j]}

ang_dis_dif = ang_dis_df.sort_values('angular distance')
print(ang_dis_dif)

In [ ]:
#for comparison (make sure the list is the same with both distances measures)

cos_dis_arr_rounded_no_repeats = np.tril(np.round(cos_distances, decimals=4))
cos_dis_df = pd.DataFrame(columns=['word1', 'word2', 'cosine distance'])
for i in range(10):
    for j in range(10):
        if cos_dis_arr_rounded_no_repeats[i,j] != 0:
            cos_dis_df.loc[len(cos_dis_df)] = {'word1': first_ten_words.iloc[i,0], 'word2': first_ten_words.iloc[j,0], 'cosine distance': cos_dis_arr_rounded_no_repeats[i,j]}

cos_dis_dif = cos_dis_df.sort_values('cosine distance')
print(cos_dis_dif)

In [ ]:
# QUESTION 2

In [9]:
start_time = time.time()

embeddings = d.iloc[:100000, 1:].to_numpy().astype('float32')
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

end_time = time.time()
print('Time for operation: ', end_time - start_time)

Time for operation:  0.830906867980957


In [10]:
print('Number of vectors in index:', index.ntotal)

Number of vectors in index: 100000


In [11]:
print('Embedding array size (MB):', embeddings.nbytes / (1024*1024))

Embedding array size (MB): 38.14697265625


In [36]:
#refers to the row index from the previous cell as the query for index.search()
#the is the number of neighbors to return
query_word = 'data'
n_neighbors_to_fetch = 31

start_time = time.time()

query_index = d[d[0]==query_word].index[0]
query_word_token = embeddings[query_index : query_index+1]
distances_to_knn, indices_of_knn = index.search(query_word_token, n_neighbors_to_fetch)

end_time = time.time()
print('Time for operation: ', end_time - start_time)

Time for operation:  0.09443473815917969


In [22]:
distances_to_knn[0]

array([ 0.      , 18.257359, 19.054846, 21.089666, 21.189434, 23.210901,
       23.637829, 24.611519, 24.932158, 25.014446, 25.301332, 25.402464,
       26.047045, 26.098978, 26.23256 , 26.469494, 27.03086 , 27.048824,
       27.206549, 27.261578, 27.412685, 27.416449, 27.439491, 27.583212,
       27.586948, 27.662615, 27.700062, 27.753094, 27.951813, 28.021767,
       28.038372], dtype=float32)

In [27]:
closest = pd.DataFrame(columns=['word','Euclidean distance'])
closest['word'] = d.iloc[indices_of_knn[0],0]
closest['Euclidean distance'] = distances_to_knn[0]
print(closest)

                 word  Euclidean distance
943              data            0.000000
447       information           18.257359
2535         analysis           19.054846
6716         tracking           21.089666
6782         database           21.189434
3605     applications           23.210901
1361          numbers           23.637829
4490         indicate           24.611519
1530          account           24.932158
1181         provided           25.014446
5950       indicating           25.301332
836         available           25.402464
15598       databases           26.047045
990          computer           26.098978
5434       comparison           26.232559
2164         accounts           26.469494
1223          systems           27.030861
11444        analyzed           27.048824
2307       statistics           27.206549
5635        indicates           27.261578
14160        analyses           27.412685
2558       electronic           27.416449
6349   identification           27

In [37]:
#exact duplicate of code from a few cells above, I just wanted to make it a new cell because everything above here works perfectly

query_word = 'data'
n_neighbors_to_fetch = 100000

start_time = time.time()

query_index = d[d[0]==query_word].index[0]
query_word_token = embeddings[query_index : query_index+1]
distances_to_knn, indices_of_knn = index.search(query_word_token, n_neighbors_to_fetch)

end_time = time.time()
print('Time for operation: ', end_time - start_time)

Time for operation:  0.11804723739624023


In [65]:
closest = pd.DataFrame(columns=['word','Euclidean distance'])
closest['word'] = d.iloc[indices_of_knn[0][1:11]][0]
closest['Euclidean distance'] = distances_to_knn[0][1:11]
print(closest)

              word  Euclidean distance
447    information           18.257359
2535      analysis           19.054846
6716      tracking           21.089666
6782      database           21.189434
3605  applications           23.210901
1361       numbers           23.637829
4490      indicate           24.611519
1530       account           24.932158
1181      provided           25.014446
5950    indicating           25.301332
